<a href="https://colab.research.google.com/github/Alice1304/Section4_project/blob/main/AI_14_%EC%B2%9C%EC%84%B8%ED%9D%AC_SECTION4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 캐글에서 데이터 가져오기

- 용량이 크기 때문에 따로 다운로드 하지 않고 캐글에서 바로 데이터를 불러오기로 함

In [1]:
# 나의 캐글 Account에서 API Token 다운 받기 = > JSON 파일 다운로드 후 캐글과 연결해주기

!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiolacheon","key":"e07252d650a626416e5bed159c28df46"}'}

In [2]:
#Kaggle API를 사용하기 위해 json 파일을 ~/.kaggle로 이동

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c word2vec-nlp-tutorial

 79% 41.0M/51.7M [00:00<00:00, 54.3MB/s]
100% 51.7M/51.7M [00:00<00:00, 62.5MB/s]


In [4]:
!ls

kaggle.json  sample_data  word2vec-nlp-tutorial.zip


In [5]:
!unzip /content/word2vec-nlp-tutorial.zip

Archive:  /content/word2vec-nlp-tutorial.zip
  inflating: labeledTrainData.tsv.zip  
  inflating: sampleSubmission.csv    
  inflating: testData.tsv.zip        
  inflating: unlabeledTrainData.tsv.zip  


In [6]:
!unzip /content/labeledTrainData.tsv.zip

Archive:  /content/labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    


In [7]:
!unzip /content/testData.tsv.zip

Archive:  /content/testData.tsv.zip
  inflating: testData.tsv            


In [8]:
!unzip /content/unlabeledTrainData.tsv.zip

Archive:  /content/unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  


In [9]:
#데이터 불러오기 

In [10]:
import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
SEED = 33

In [11]:
train = pd.read_csv('/content/labeledTrainData.tsv', delimiter='\t')
test = pd.read_csv('/content/testData.tsv', delimiter='\t')

In [12]:
unlabeled_train = pd.read_csv('/content/unlabeledTrainData.tsv', delimiter='\t', error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [13]:
train.shape, test.shape, unlabeled_train.shape

((25000, 3), (25000, 2), (49998, 2))

In [14]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [15]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [16]:
 unlabeled_train.head()

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


## 2. 전처리

In [17]:
#html텍스트가 포함되어 있어 삭제
#불용어 제거

from bs4 import BeautifulSoup
import re

In [18]:
sample = train['review'][0]

In [19]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [20]:
soup = BeautifulSoup(sample, 'html.parser')

In [21]:
soup.text  #html삭제

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [22]:
clean = re.sub('[^a-zA-Z]', ' ', soup.text)  #알파벳을 제외한 문자 모두 공백처리

In [23]:
clean

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [24]:
clean.lower()

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

In [25]:
!pip install nltk
import nltk 
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
from nltk.corpus import stopwords

In [27]:
eng_stopwords = stopwords.words('english')

## 3. lemmatizer

In [28]:
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
#단어를 원형으로 변환해주는 라이브러리

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [29]:
lemmatizer = WordNetLemmatizer()

In [30]:
lemmatizer.lemmatize('runs')

'run'

In [31]:
def process_lemma(sentence):
  return [lemmatizer.lemmatize(word, 'v') for word in sentence]

In [32]:
# 전처리 과정을 하나의 함수로 출력하기
# 불용어 제외하고 출력하기 & 동사의 원형으로 변환 

def preprocessing(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  clean = re.sub('[^a-zA-Z]', ' ', soup.text) 
  clean = clean.lower()
  clean = [word for word in clean.split() if word not in eng_stopwords]
  clean = process_lemma(clean)

  return ' '.join(clean)  #띄어쓰기 넣어서 조인하기 


In [33]:
all_review = pd.concat([train['review'], unlabeled_train['review'],test['review']])

In [34]:
all_review_clean = all_review.apply(preprocessing)

In [35]:
all_review_clean.head() 

0    stuff go moment mj start listen music watch od...
1    classic war worlds timothy hines entertain fil...
2    film start manager nicholas bell give welcome ...
3    must assume praise film greatest film opera ev...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

In [36]:
all_review_clean.shape

(99998,)

## 4. Tokenizer
- 단어를 하나씩 떼어서 단어사전 만들어주기

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [38]:
tokenizer = Tokenizer(oov_token='<OOV>')

In [39]:
tokenizer.fit_on_texts(all_review_clean)

In [40]:
len(tokenizer.word_index)

126312

In [41]:
tokenizer.word_index['give']

32

In [42]:
for i, word in enumerate(tokenizer.word_index): #단어사전만들기 
  if i > 20:
    break
  print(word, tokenizer.word_index[word])

<OOV> 1
film 2
movie 3
one 4
make 5
like 6
see 7
get 8
time 9
good 10
character 11
go 12
watch 13
even 14
would 15
think 16
story 17
really 18
well 19
show 20
look 21


In [43]:
train_sentences = all_review_clean[:len(train)]
test_sentences = all_review_clean[-len(test):]

In [44]:
train_sentences.shape, test_sentences.shape


((25000,), (25000,))

In [45]:
train_sequences = tokenizer.texts_to_sequences(train_sentences) #문자형_숫자형 치환
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [46]:
train_sequences[0]

[397,
 12,
 463,
 11594,
 83,
 931,
 127,
 13,
 895,
 507,
 13,
 21106,
 13,
 19437,
 179,
 46,
 8,
 639,
 2250,
 66,
 16,
 18,
 469,
 3273,
 179,
 5,
 188,
 643,
 2110,
 1155,
 19437,
 58,
 4431,
 58,
 258,
 2,
 240,
 12,
 7,
 349,
 1643,
 255,
 1145,
 550,
 11594,
 59,
 773,
 2039,
 29,
 471,
 550,
 593,
 26,
 4231,
 1924,
 1032,
 175,
 420,
 1453,
 782,
 2209,
 6,
 11594,
 459,
 12,
 613,
 37,
 170,
 116,
 146,
 11594,
 34889,
 9296,
 5,
 3,
 11594,
 109,
 15,
 25,
 5,
 109,
 198,
 18,
 253,
 727,
 258,
 2,
 114,
 339,
 83,
 141,
 7788,
 3475,
 1502,
 311,
 781,
 6909,
 526,
 9123,
 785,
 593,
 1370,
 46,
 11594,
 242,
 26,
 558,
 11594,
 9785,
 505,
 12451,
 781,
 6909,
 11,
 3763,
 46,
 27,
 24,
 2666,
 593,
 413,
 8743,
 179,
 724,
 11594,
 127,
 64,
 469,
 94,
 6,
 11594,
 90,
 419,
 1905,
 130,
 1523,
 2147,
 311,
 29,
 68,
 113,
 3929,
 3388,
 36,
 2,
 22089,
 26,
 311,
 516,
 843,
 613,
 43,
 4,
 129,
 152,
 518,
 130,
 630,
 890,
 1120,
 423,
 55,
 1131,
 107,
 3,
 27,
 6,
 

In [47]:
#문장의 길이가 다르기 때문에 문장의 길이를 맞춰주는 작업 진행 

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [48]:
MAX_LENGTH = 150 #임의로 설정

In [49]:
train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, truncating='post', padding='post') 
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, truncating='post', padding='post')
#truncating = 150 단어를 넘어갈 경우 뒷부분 자르기, padding = 단어가 부족할 경우 뒤에 채워주기 

In [50]:
train_padded.shape,test_padded.shape

((25000, 150), (25000, 150))

## 5. train test split 



In [51]:
from sklearn.model_selection import train_test_split

In [52]:
train_labels = train['sentiment']

In [53]:
x_train, x_valid, y_train, y_valid = train_test_split(train_padded, train_labels, stratify=train_labels, test_size = 0.1, random_state=SEED)

## 6. Model

- 차원을 줄여주기 위해 임베딩 진행

In [54]:
EMBEDDING_DIM= 300
VOCAB_SIZE = len(tokenizer.word_index) + 1 #패딩이 0으로 들어가있기 때문에 +1 더해주기 

In [55]:
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [56]:
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(128)),
    Dropout(0.25),
    Dense(32,activation='relu'),
    Dense(1, activation='sigmoid')
    ])

In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          37893900  
                                                                 
 bidirectional (Bidirectiona  (None, 150, 256)         439296    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dense_1 (Dense)             (None, 1)                 3

In [58]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [59]:
checkpoint_path = 'tmp/checkpoint.ckpt'
checkpoint= ModelCheckpoint(filepath = checkpoint_path,
                            save_best_only=True,
                            save_weight_only=True,
                            monitor='val_loss',
                            verbose=1,
                            )

In [60]:
model.fit(x_train,y_train,
          validation_data=(x_valid, y_valid),
          batch_size=128,
          epochs=5,
          callbacks=[checkpoint])

Epoch 1/5
176/176 [==============================] - ETA: 0s - loss: 0.3937 - acc: 0.8133
Epoch 1: val_loss improved from inf to 0.29469, saving model to tmp/checkpoint.ckpt


176/176 [==============================] - 50s 225ms/step - loss: 0.3937 - acc: 0.8133 - val_loss: 0.2947 - val_acc: 0.8800
Epoch 2/5
176/176 [==============================] - ETA: 0s - loss: 0.1546 - acc: 0.9449
Epoch 2: val_loss did not improve from 0.29469
176/176 [==============================] - 13s 76ms/step - loss: 0.1546 - acc: 0.9449 - val_loss: 0.3438 - val_acc: 0.8568
Epoch 3/5
176/176 [==============================] - ETA: 0s - loss: 0.0718 - acc: 0.9775
Epoch 3: val_loss did not improve from 0.29469
176/176 [==============================] - 13s 76ms/step - loss: 0.0718 - acc: 0.9775 - val_loss: 0.3733 - val_acc: 0.8608
Epoch 4/5
176/176 [==============================] - ETA: 0s - loss: 0.0563 - acc: 0.9824
Epoch 4: val_loss did not improve from 0.29469
176/176 [==============================] - 14s 77ms/step - loss: 0.0563 - acc: 0.9824 - val_loss: 0.4310 - val_acc: 0.8528
Epoch 5/5
176/176 [==============================] - ETA: 0s - loss: 0.0590 - acc: 0.9808
Epoch 

In [61]:
model.load_weights(checkpoint_path)

In [62]:
model.evaluate(x_valid,y_valid)

79/79 [==============================] - 1s 16ms/step - loss: 0.2947 - acc: 0.8800


[0.29468703269958496, 0.8799999952316284]

In [63]:
prediction = model.predict(test_padded)

In [64]:
prediction

array([[0.9708417 ],
       [0.06348298],
       [0.26860812],
       ...,
       [0.21581815],
       [0.96817464],
       [0.5768248 ]], dtype=float32)

In [65]:
prediction[prediction >= 0.5] = 1
prediction[prediction < 0.5] = 0

In [66]:
prediction

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

## 7. 실제 데이터를 가지고 예측해보기 

In [67]:
def prediction(new_sentence):
  new_sentence = re.sub('[^a-zA-Z]', ' ', new_sentence)
  new_sentence = new_sentence.lower()
  new_sentence = [word for word in new_sentence.split() if word not in eng_stopwords]
  new_sentence = ' '.join(new_sentence)
  output = new_sentence
  tokenizer.fit_on_texts(output)
  encoded = tokenizer.texts_to_sequences(output) #인코딩
  pad_new = pad_sequences(encoded, maxlen = MAX_LENGTH)
  score = model.predict(pad_new)[1] # 예측
  if(score > 0.5):
    print("{0}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{0}% 확률로 부정 리뷰입니다.\n".format(100-score * 100))


In [68]:
review_good = """
Mar 24, 2021
I'm being 100% honest with you whoever reads this review I think the movie was good. even though I wished there were more action scenes in the movie. but overall I think it is good
"""

In [69]:
prediction(review_1)

[56.952686]% 확률로 부정 리뷰입니다.



In [70]:
review_bad = """

Aug 20, 2020
you know you touched rock bottom when you willingly watch transformers 5. what a piece of garbage -Dialogue as if the writers hadn't talk to a kid in their life (or another human being for that matter) -aspect ratio changing EVERY SINGLE SHOT, not every scene; EVERY. SHOT. -camera movements without reason -cuts and framing that would be nice if it was a PARODY of an action movie and that would make anyone with eyes cringe. -random vignetting like they don't know how to use a fucking lens -no character devel
"""

In [71]:
prediction(review_2)

[56.904377]% 확률로 부정 리뷰입니다.

